In [3]:
# to import configured input and output paths
import configparser
# Dataframe building and manipulation
import pandas as pd
import numpy as np
# to temporarily add the src path to the system to import the src modules
import sys as sys

'''Model Building'''
#PreProcessing
from sklearn.preprocessing import StandardScaler

#Model Stuff
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [3]:
# import my config.ini file
config = configparser.ConfigParser()
config.read("../src/config.ini")
# add src path to system paths
sys.path.append(config['paths']['src_path'])

In [4]:
train = pd.read_csv(config['paths']['data_path'] + 'training_set.csv')
test = pd.read_csv(config['paths']['data_path'] + 'testing_set.csv')